In [ ]:
import numpy as np
import cv2
import os

def simulate_realistic_tennis_trajectory(num_points=100, dt=0.03):
    """
    Simulates realistic tennis ball trajectory with physics:
    - Gravity
    - Air resistance  
    - Spin effects (Magnus force)
    - Court bounces
    """
    # Tennis ball parameters
    mass = 0.057  # kg
    radius = 0.033  # m
    Cd = 0.47  # drag coefficient
    air_density = 1.225  # kg/m³
    restitution = 0.7  # bounce coefficient
    friction = 0.3  # court friction
    
    # Initial conditions - start from center of court
    positions = []
    velocities = []
    
    # Starting position - center of court, good height for visibility
    x0, y0, z0 = 0.0, 0.0, 2.5  # Start from center of court at net height
    
    # Initial velocity - hit toward one side of court
    v0_x = 12.0   # m/s forward (toward positive X)
    v0_y = 4.0    # m/s toward one side
    v0_z = -2.0   # m/s downward angle
    
    # Spin (rad/s) - moderate topspin
    spin_x = 0.0    # no sidespin
    spin_y = 0.0    # no axis spin
    spin_z = -20.0  # moderate topspin
    
    # Initialize with explicit float arrays
    pos = np.array([x0, y0, z0], dtype=np.float64)
    vel = np.array([v0_x, v0_y, v0_z], dtype=np.float64)
    spin = np.array([spin_x, spin_y, spin_z], dtype=np.float64)
    
    positions.append(pos.copy())
    velocities.append(vel.copy())
    
    for i in range(num_points - 1):
        # Current speed
        speed = np.linalg.norm(vel)
        
        if speed < 0.1:  # Ball stopped
            break
            
        # Air resistance force
        A = np.pi * radius**2  # cross-sectional area
        drag_force = -0.5 * air_density * Cd * A * speed * vel
        
        # Magnus force (spin effect)
        omega = spin
        magnus_force = 0.5 * air_density * A * speed * np.cross(omega, vel)
        
        # Total acceleration
        gravity = np.array([0.0, 0.0, -9.81], dtype=np.float64)
        a_drag = drag_force / mass
        a_magnus = magnus_force / mass
        acceleration = gravity + a_drag + a_magnus
        
        # Update velocity and position (Euler integration)
        vel = vel + acceleration * dt
        pos = pos + vel * dt
        
        # Spin decay due to air resistance
        spin = spin * 0.999
        
        # Check for court bounce
        if pos[2] <= 0 and vel[2] < 0:  # Hit ground
            # Bounce physics
            vel[2] = -vel[2] * restitution  # vertical bounce
            vel[0] = vel[0] * (1 - friction)  # horizontal friction
            vel[1] = vel[1] * (1 - friction)
            pos[2] = 0.01  # Slightly above ground
            
            # Spin changes after bounce
            spin[2] = spin[2] * 0.8  # topspin reduces
            
        # Check if ball went too far out of bounds
        if abs(pos[0]) > 15 or abs(pos[1]) > 15:
            break
            
        positions.append(pos.copy())
        velocities.append(vel.copy())
    
    return np.array(positions), np.array(velocities)

def generate_camera_poses():
    """
    Generate three cameras with specific positioning using corrected rotation matrix:
    Camera 1: Center view
    Camera 2: Left side, looking toward origin, tilting down 
    Camera 3: Right side, looking toward origin, tilting down
    """
    camera_poses = []
    
    # Camera positioning parameters
    camera_height = 3.0      # Higher for better downward view
    distance_from_origin = 6.0
    lateral_separation = 4.0
    
    # Define cameras by position and target
    camera_configs = [
        {
            'name': 'Camera 1 (Center)',
            'position': np.array([0.0, -distance_from_origin, camera_height]),
            'target': np.array([0.0, 0.0, 0.5])  # Look at court center, slightly above ground
        },
        {
            'name': 'Camera 2 (Left)',
            'position': np.array([-lateral_separation, -distance_from_origin, camera_height]),
            'target': np.array([0.0, 0.0, 0.0])  # Look at origin on ground
        },
        {
            'name': 'Camera 3 (Right)',
            'position': np.array([lateral_separation, -distance_from_origin, camera_height]),
            'target': np.array([0.0, 0.0, 0.0])  # Look at origin on ground
        }
    ]
    
    for i, config in enumerate(camera_configs):
        camera_pos = config['position']
        target = config['target']
        
        # Calculate look vector (from camera to target)
        look_vector = target - camera_pos
        look_vector = look_vector / np.linalg.norm(look_vector)
        
        # Build camera coordinate system using standard computer vision convention
        # Camera looks along +Z axis (not -Z as I had before)
        forward = look_vector  # This is the direction camera looks (+Z in camera coords)
        
        # Right vector (camera +X axis)
        world_up = np.array([0, 0, 1])
        right =  np.cross(forward, world_up)
        if np.linalg.norm(right) < 1e-6:
            # Looking straight up/down, use different reference
            right = np.array([1, 0, 0])
        else:
            right = right / np.linalg.norm(right)
        
        # Up vector (camera +Y axis)
        up = - np.cross(right, forward)
        
        # Create rotation matrix (camera to world)
        R_cam_to_world = np.column_stack([right, up, forward])
        
        # Invert to get world to camera
        R_world_to_cam = R_cam_to_world.T
        
        # Translation vector (position of world origin in camera coordinates)
        t_world_to_cam = -R_world_to_cam @ camera_pos
        
        # Store camera pose
        camera_pose = np.eye(4)
        camera_pose[:3, :3] = R_world_to_cam
        camera_pose[:3, 3] = t_world_to_cam
        
        camera_poses.append(camera_pose)
        
        # Calculate and verify angles
        pan_angle = np.degrees(np.arctan2(look_vector[1], look_vector[0]))
        tilt_angle = np.degrees(np.arcsin(look_vector[2]))
        
        print(f"{config['name']}: pos={camera_pos}")
        print(f"  Target: {target}")
        print(f"  Look vector: {look_vector}")
        print(f"  Pan: {pan_angle:.1f}°, Tilt: {tilt_angle:.1f}°")
        
        # Test projection of key points to verify camera is working
        test_points = np.array([
            [0, 0, 0],      # Origin
            [0, 0, 1],      # Above origin  
            [-3, 0, 0],     # Left side
            [3, 0, 0],      # Right side
            camera_pos + 2 * look_vector,  # Point in front of camera
        ])
        
        K = get_camera_intrinsics()
        test_2d, test_depths = project_points_simple(K, R_world_to_cam, t_world_to_cam, test_points)
        
        print(f"  Test projections:")
        for j, (pt_3d, pt_2d, depth) in enumerate(zip(test_points, test_2d, test_depths)):
            print(f"    {pt_3d} -> ({pt_2d[0]:.0f}, {pt_2d[1]:.0f}), depth {depth:.1f}")
        print()
    
    return camera_poses

def get_camera_intrinsics(image_size=(640, 480), focal_length=400):
    """Generate camera intrinsic matrix with tighter field of view for focused court coverage"""
    cx, cy = image_size[0] / 2, image_size[1] / 2
    K = np.array([
        [focal_length, 0, cx],
        [0, focal_length, cy],
        [0, 0, 1]
    ])
    return K

def project_points_simple(K, R, t, points_3d):
    """Simple projection using standard camera equations with detailed debugging"""
    # Transform to camera coordinates
    points_cam = (R @ points_3d.T).T + t.reshape(1, 3)
    
    points_2d = np.zeros((len(points_3d), 2))
    depths = np.zeros(len(points_3d))
    
    for i, (x, y, z) in enumerate(points_cam):
        depths[i] = z
        if z > 0.1:  # Point in front of camera
            u = K[0,0] * x/z + K[0,2]
            v = K[1,1] * y/z + K[1,2]
            points_2d[i] = [u, v]
        else:
            points_2d[i] = [np.nan, np.nan]
    
    return points_2d, depths

def create_tennis_court_3d():
    """Create 3D tennis court geometry with smaller, more visible dimensions"""
    # Smaller court for better camera visibility
    court_length = 12.0  # meters (reduced from full court)
    court_width = 6.0    # meters (reduced from full court)
    
    # Court positioned around origin for better camera views
    # Court extends from -court_length/2 to +court_length/2 in X
    # Court extends from -court_width/2 to +court_width/2 in Y
    
    # Court corners (ground level z=0)
    corners = [
        [-court_length/2, -court_width/2, 0],  # Back left
        [court_length/2, -court_width/2, 0],   # Front left  
        [court_length/2, court_width/2, 0],    # Front right
        [-court_length/2, court_width/2, 0],   # Back right
    ]
    
    # Court lines for tennis court
    lines = []
    
    # Court boundary (rectangle)
    boundary_lines = [
        [[-court_length/2, -court_width/2, 0], [-court_length/2, court_width/2, 0]],  # Left baseline
        [[court_length/2, -court_width/2, 0], [court_length/2, court_width/2, 0]],    # Right baseline
        [[-court_length/2, -court_width/2, 0], [court_length/2, -court_width/2, 0]],  # Bottom sideline
        [[-court_length/2, court_width/2, 0], [court_length/2, court_width/2, 0]],    # Top sideline
    ]
    
    # Net line (center)
    net_line = [[0, -court_width/2, 0], [0, court_width/2, 0]]
    
    # Service lines (3m from net on each side)
    service_distance = 3.0
    service_lines = [
        [[-service_distance, -court_width/2, 0], [-service_distance, court_width/2, 0]],  # Left service line
        [[service_distance, -court_width/2, 0], [service_distance, court_width/2, 0]],    # Right service line
        [[-service_distance, 0, 0], [service_distance, 0, 0]],  # Center service line
    ]
    
    # Combine all lines
    all_lines = boundary_lines + [net_line] + service_lines
    
    return np.array(corners), np.array(all_lines)

def create_perspective_tennis_court(image_size, K, R, t):
    """Create tennis court background with proper perspective from camera viewpoint"""
    image = np.zeros((image_size[1], image_size[0], 3), dtype=np.uint8)
    
    # Sky gradient (upper portion only)
    sky_height = int(image_size[1] * 0.5)  # Sky takes up top 50%
    for y in range(sky_height):
        intensity = 180 + int((y / sky_height) * 55)
        blue = min(255, intensity + 75)
        green = min(255, intensity + 26) 
        red = min(255, intensity - 45)
        image[y, :] = [blue, green, red]  # BGR format
    
    # Get court geometry
    court_corners, court_lines = create_tennis_court_3d()
    
    # Project court corners to 2D
    court_2d, court_depths = project_points_simple(K, R, t, court_corners)
    
    # Debug: Print court projection info
    print(f"Court corners 3D: {court_corners}")
    print(f"Court depths: {court_depths}")
    print(f"Court 2D: {court_2d}")
    
    # Draw court surface (green quad) if all corners are visible
    if np.all(court_depths > 0.1):  # All corners in front of camera
        # Check if points are reasonable
        valid_2d = ~np.isnan(court_2d).any(axis=1)
        if np.all(valid_2d):
            court_2d_int = court_2d.astype(np.int32)
            court_color = (34, 139, 34)  # BGR: Forest green
            
            # Create and fill court polygon
            cv2.fillPoly(image, [court_2d_int], court_color)
            print(f"Drew court polygon with corners: {court_2d_int}")
            
            # Draw court boundary (white outline)
            line_color = (255, 255, 255)
            cv2.polylines(image, [court_2d_int], True, line_color, 3)
        else:
            print("Some court corners have invalid 2D projections")
    else:
        print("Some court corners are behind camera or too close")
    
    # Draw all court lines
    line_color = (255, 255, 255)  # White lines
    for i, line_pair in enumerate(court_lines):
        # Project line endpoints
        line_2d, line_depths = project_points_simple(K, R, t, line_pair)
        
        print(f"Line {i}: 3D {line_pair} -> 2D {line_2d}, depths {line_depths}")
        
        # Draw line if both endpoints are in front of camera
        if np.all(line_depths > 0.1) and not np.isnan(line_2d).any():
            pt1 = tuple(line_2d[0].astype(int))
            pt2 = tuple(line_2d[1].astype(int))
            
            # Draw line regardless of bounds (will be clipped by OpenCV)
            cv2.line(image, pt1, pt2, line_color, 2)
            print(f"Drew line from {pt1} to {pt2}")
        else:
            print(f"Skipped line {i}: behind camera or invalid")
    
    return image

def create_realistic_tennis_view(image_size, K, R, t, points_2d, depths, velocities, ball_radius_world=0.033):
    """Create realistic tennis camera view with perspective-correct court and distance-based ball sizing"""
    # Create perspective-correct court background
    image = create_perspective_tennis_court(image_size, K, R, t)
    
    valid_points = ~np.isnan(points_2d).any(axis=1) & (depths > 0)
    
    print(f"Debug ball sizing - showing ALL balls:")
    print(f"Total valid balls to draw: {np.sum(valid_points)}")
    
    for i, ((x, y), depth, vel) in enumerate(zip(points_2d[valid_points], depths[valid_points], velocities[valid_points])):
        if np.isnan(depth) or depth <= 0:
            continue
            
        # Calculate ball size with proper precision
        ball_radius_pixels_float = (ball_radius_world * K[0,0]) / depth
        
        # Apply distance-based minimum sizes (keeping as float until final conversion)
        if depth <= 2.0:        # Very close
            ball_radius_pixels_float = max(ball_radius_pixels_float, 12.0)
        elif depth <= 5.0:      # Medium distance  
            ball_radius_pixels_float = max(ball_radius_pixels_float, 6.0)
        elif depth <= 10.0:     # Far
            ball_radius_pixels_float = max(ball_radius_pixels_float, 3.0)
        else:                   # Very far
            ball_radius_pixels_float = max(ball_radius_pixels_float, 2.0)
            
        # Maximum limit to prevent huge balls when very close
        ball_radius_pixels_float = min(ball_radius_pixels_float, 25.0)
        
        # Convert to integer only at the very end
        ball_radius_pixels = int(round(ball_radius_pixels_float))
        
        # Debug: Print ALL ball calculations to find the anomaly
        print(f"  Ball {i}: depth={depth:.1f}m, raw_size={ball_radius_pixels_float:.2f}px, final={ball_radius_pixels}px")
        
        x, y = int(x), int(y)
        
        # Tennis ball color - BGR format
        ball_color = (0, 255, 255)  # BGR: bright yellow tennis ball
        
        if 0 <= x < image_size[0] and 0 <= y < image_size[1]:
            # Main ball
            cv2.circle(image, (x, y), ball_radius_pixels, ball_color, -1)
            
            # Highlight for 3D effect (white) - scale with ball size
            highlight_radius = max(1, ball_radius_pixels // 3)
            highlight_x = x - ball_radius_pixels // 3
            highlight_y = y - ball_radius_pixels // 3
            cv2.circle(image, (highlight_x, highlight_y), highlight_radius, (255, 255, 255), -1)
            
            # Add depth info as small text next to ball
            depth_text = f"{depth:.1f}m"
            font_scale = 0.3
            text_color = (255, 255, 255)  # White text
            cv2.putText(image, depth_text, (x + ball_radius_pixels + 2, y), 
                       cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, 1)
    
    return image

def create_realistic_time_sequence(image_size, K, R, t, points_2d, depths, velocities, num_frames=15, ball_radius_world=0.033):
    """Create realistic time sequence with perspective-correct court and distance-based ball sizing"""
    frames = []
    
    # Filter valid points first
    valid_points = ~np.isnan(points_2d).any(axis=1) & (depths > 0)
    valid_2d = points_2d[valid_points]
    valid_depths = depths[valid_points]
    valid_velocities = velocities[valid_points]
    
    print(f"Debug: Total points: {len(points_2d)}, Valid points: {len(valid_2d)}")
    
    if len(valid_2d) == 0:
        print("No valid points found!")
        return [create_perspective_tennis_court(image_size, K, R, t)]
    
    print(f"Debug: Valid points X range: [{valid_2d[:,0].min():.1f}, {valid_2d[:,0].max():.1f}]")
    print(f"Debug: Valid points Y range: [{valid_2d[:,1].min():.1f}, {valid_2d[:,1].max():.1f}]")
    print(f"Debug: Depth range: [{valid_depths.min():.1f}, {valid_depths.max():.1f}]")
    
    # Create frames
    indices = np.linspace(0, len(valid_2d) - 1, num_frames, dtype=int)
    
    for frame_idx, ball_idx in enumerate(indices):
        # Create perspective-correct court background
        image = create_perspective_tennis_court(image_size, K, R, t)
        
        # Calculate realistic trail length based on ball speed
        current_speed = np.linalg.norm(valid_velocities[ball_idx])
        trail_length = int(min(8, max(3, current_speed / 4)))
        
        # Draw trail with distance-based sizing
        for trail_step in range(trail_length):
            trail_idx = ball_idx - trail_step
            if trail_idx < 0:
                break
                
            x, y = int(valid_2d[trail_idx, 0]), int(valid_2d[trail_idx, 1])
            depth = valid_depths[trail_idx]
            
            # Draw trail if visible
            if 0 <= x < image_size[0] and 0 <= y < image_size[1] and depth > 0:
                # FIXED: Trail size also uses proper precision
                trail_base_radius_float = (ball_radius_world * K[0,0]) / depth
                trail_base_radius = int(round(trail_base_radius_float))
                trail_base_radius = max(1, min(trail_base_radius, 15))
                
                # Trail fades and gets smaller
                fade = (trail_length - trail_step) / trail_length
                trail_radius = max(1, int(round(trail_base_radius * fade * 0.6)))  # Smaller than main ball
                trail_alpha = int(120 * fade)
                trail_color = (trail_alpha, trail_alpha, trail_alpha)  # Gray trail
                cv2.circle(image, (x, y), trail_radius, trail_color, -1)
        
        # Draw current ball with FIXED distance-based sizing
        x, y = valid_2d[ball_idx]
        depth = valid_depths[ball_idx]
        velocity = valid_velocities[ball_idx]
        
        if depth > 0:
            # FIXED: Ball size calculation with proper precision
            ball_radius_pixels_float = (ball_radius_world * K[0,0]) / depth
            
            # Distance-based size clamping for realism (keep as float)
            if depth <= 2.0:        # Very close
                ball_radius_pixels_float = max(ball_radius_pixels_float, 12.0)
            elif depth <= 5.0:      # Medium distance  
                ball_radius_pixels_float = max(ball_radius_pixels_float, 6.0)
            elif depth <= 10.0:     # Far
                ball_radius_pixels_float = max(ball_radius_pixels_float, 3.0)
            else:                   # Very far
                ball_radius_pixels_float = max(ball_radius_pixels_float, 2.0)
                
            # Maximum limit
            ball_radius_pixels_float = min(ball_radius_pixels_float, 25.0)
            
            # Convert to integer only at the end
            ball_radius_pixels = int(round(ball_radius_pixels_float))
            
            x, y = int(x), int(y)
            
            if 0 <= x < image_size[0] and 0 <= y < image_size[1]:
                # Tennis ball color - BGR format  
                ball_color = (0, 255, 255)  # BGR: bright yellow tennis ball
                
                # Main ball
                cv2.circle(image, (x, y), ball_radius_pixels, ball_color, -1)
                
                # Highlight (white) - scale with ball size
                highlight_radius = max(1, ball_radius_pixels // 3)
                cv2.circle(image, (x - ball_radius_pixels//3, y - ball_radius_pixels//3), 
                          highlight_radius, (255, 255, 255), -1)
        
        # Add debug info with distance information
        speed = np.linalg.norm(velocity)
        speed_text = f"Speed: {speed:.1f} m/s"
        cv2.putText(image, speed_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(image, f"Frame {frame_idx+1}/{num_frames}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Add ball position and depth info
        depth_text = f"Ball: ({x}, {y}), depth: {depth:.1f}m, size: {ball_radius_pixels}px"
        cv2.putText(image, depth_text, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        frames.append(image)
    
    return frames

def simulate_and_save(output_path="data/simulated/synthetic_scene.npz", image_size=(640, 480)):
    """Generate realistic tennis ball simulation with fixed perspective"""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    os.makedirs("data/simulated/camera_views", exist_ok=True)
    os.makedirs("data/simulated/time_sequence", exist_ok=True)

    # Generate realistic tennis trajectory (SAME trajectory for all cameras)
    points_3d, velocities = simulate_realistic_tennis_trajectory()
    print(f"Tennis trajectory: {len(points_3d)} points")
    print(f"3D range: x[{points_3d[:,0].min():.1f}, {points_3d[:,0].max():.1f}], "
          f"y[{points_3d[:,1].min():.1f}, {points_3d[:,1].max():.1f}], "
          f"z[{points_3d[:,2].min():.1f}, {points_3d[:,2].max():.1f}]")
    print(f"Speed range: {np.linalg.norm(velocities, axis=1).min():.1f} - {np.linalg.norm(velocities, axis=1).max():.1f} m/s")
    
    # Debug: Print ALL 3D positions to find anomalies
    print(f"\nALL ball 3D positions:")
    for i in range(len(points_3d)):
        pos = points_3d[i]
        print(f"  Point {i}: [{pos[0]:6.2f}, {pos[1]:6.2f}, {pos[2]:6.2f}]")
    
    # Camera parameters
    intrinsics = get_camera_intrinsics(image_size=image_size)
    camera_poses = generate_camera_poses()

    projections_2d = []
    all_depths = []

    for i, camera_pose in enumerate(camera_poses):
        R = camera_pose[:3, :3]
        t = camera_pose[:3, 3]
        
        print(f"\n=== Processing Camera {i+1} ===")
        camera_pos = np.array([0.0, -5.0, 3.5])  # Camera 1 position for reference
        if i == 1:
            camera_pos = np.array([-4.0, -3.5, 3.5])  # Camera 2 position
        elif i == 2:
            camera_pos = np.array([4.0, -3.5, 3.5])   # Camera 3 position
            
        print(f"Camera position: {camera_pos}")
        
        # Project SAME 3D trajectory to this camera's 2D view
        pts_2d, depths = project_points_simple(intrinsics, R, t, points_3d)
        
        # Debug: Show transformations for ALL points to find the close one
        print(f"Sample transformations for camera {i+1}:")
        for j in range(len(points_3d)):
            world_pt = points_3d[j]
            cam_pt = R @ world_pt + t
            if cam_pt[2] > 0:
                # Calculate 3D distance from camera to ball
                distance_3d = np.linalg.norm(world_pt - camera_pos)
                pixel_pt = pts_2d[j]
                print(f"  Point {j}: World{world_pt} -> depth={depths[j]:.1f}m, 3D_dist={distance_3d:.1f}m")
                
                # Flag unusual points
                if depths[j] < 2.0 or distance_3d < 3.0:
                    print(f"    *** ANOMALY: Very close ball! ***")
        
        # Statistics
        valid_points = ~np.isnan(pts_2d).any(axis=1) & (depths > 0)
        pts_2d_valid = pts_2d[valid_points]
        depths_valid = depths[valid_points]
        
        if len(pts_2d_valid) > 0:
            x_min, x_max = pts_2d_valid[:,0].min(), pts_2d_valid[:,0].max()
            y_min, y_max = pts_2d_valid[:,1].min(), pts_2d_valid[:,1].max()
            depth_min, depth_max = depths_valid.min(), depths_valid.max()
            
            in_bounds = ((pts_2d_valid[:,0] >= 0) & (pts_2d_valid[:,0] < image_size[0]) & 
                        (pts_2d_valid[:,1] >= 0) & (pts_2d_valid[:,1] < image_size[1]))
            in_bounds_count = np.sum(in_bounds)
            
            print(f"  2D range: x[{x_min:.1f}, {x_max:.1f}], y[{y_min:.1f}, {y_max:.1f}]")
            print(f"  Depth range: [{depth_min:.1f}, {depth_max:.1f}]m")
            print(f"  Visible points: {in_bounds_count}/{len(pts_2d_valid)} in image bounds")
        
        projections_2d.append(pts_2d)
        all_depths.append(depths)

        # Create realistic tennis view with perspective-correct court
        realistic_view = create_realistic_tennis_view(image_size, intrinsics, R, t, pts_2d, depths, velocities)
        cv2.imwrite(f"data/simulated/camera_views/cam_{i+1}_tennis_view.png", realistic_view)
        
        # Create realistic time sequence with perspective-correct court
        print(f"Generating frames for camera {i+1}...")
        sequence_frames = create_realistic_time_sequence(
            image_size, intrinsics, R, t, pts_2d, depths, velocities, num_frames=15)
        for frame_idx, frame in enumerate(sequence_frames):
            cv2.imwrite(f"data/simulated/time_sequence/cam_{i+1}_frame_{frame_idx+1:02d}.png", frame)

    # Save all data
    np.savez_compressed(output_path,
                        points_3d=points_3d,
                        velocities=velocities,
                        intrinsics=intrinsics,
                        projections_2d=np.array(projections_2d, dtype=object),
                        depths=np.array(all_depths, dtype=object),
                        camera_poses=np.array(camera_poses))
    print(f"\Tennis simulation saved to: {output_path}")
    print(f"Tennis court views saved to: data/simulated/camera_views/")
    print(f"Time sequence frames saved to: data/simulated/time_sequence/")



In [ ]:
simulate_and_save()